In [1]:
from google.auth import default

credentials, project = default()

In [11]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv

load_dotenv()

from src.config_loader import config
from src.utils import get_first_n_items
from src.gcs_storage import read_from_gcs, get_translated_phrases_path
from src.audio_generation import upload_phrases_audio_to_gcs

print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "LM2000"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found existing FFmpeg paths:
  C:\Users\andyl\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-7.1.1-full_build\bin
Swedish


# Phrase Audio

## Storage overview

Save MP3 files of audio with the filenames of the phrase keys based on clean_filename

multimedia/audio/phrases/french/normal/this_is_a_phrase.mp3
multimedia/audio/phrases/french/slow/this_is_a_phrase.mp3

This allows us to quickly retrieve a phrase based on the bucket name and the phrase key, as well as modify individual phrases for later correction

In [3]:
language_name = config.TARGET_LANGUAGE_NAME.lower()
print(language_name)
config.get_voice_models()

swedish


(VoiceInfo(name='en-GB-Neural2-B', provider=<VoiceProvider.GOOGLE: 'google'>, voice_id='en-GB-Neural2-B', language_code='en-GB'),
 VoiceInfo(name='sv-SE-SofieNeural', provider=<VoiceProvider.AZURE: 'azure'>, voice_id='sv-SE-SofieNeural', language_code='sv-SE'),
 VoiceInfo(name='sv-SE-MattiasNeural', provider=<VoiceProvider.AZURE: 'azure'>, voice_id='sv-SE-MattiasNeural', language_code='sv-SE'))

In [7]:
# get translations

phrase_translations = read_from_gcs(
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_path=get_translated_phrases_path(collection=COLLECTION),
)

In [8]:
phrase_translations["the_drugs_reaction_was_relatively_mild_thank_goodness"]

{'english': "The drug's reaction was relatively mild, thank goodness.",
 'swedish': 'Medicinen gav bara en mild reaktion, tur det.',
 'modified': True,
 'wiktionary_links': '<a href="https://en.wiktionary.org/wiki/medicinen#Swedish" target="_blank" rel="noopener">Medicinen</a> <a href="https://en.wiktionary.org/wiki/gav#Swedish">gav</a> <a href="https://en.wiktionary.org/wiki/bara#Swedish">bara</a> <a href="https://en.wiktionary.org/wiki/en#Swedish">en</a> <a href="https://en.wiktionary.org/wiki/mild#Swedish" target="_blank" rel="noopener">mild</a> <a href="https://en.wiktionary.org/wiki/reaktion#Swedish" target="_blank" rel="noopener">reaktion,</a> <a href="https://en.wiktionary.org/wiki/tur#Swedish">tur</a> <a href="https://en.wiktionary.org/wiki/det#Swedish">det</a>'}

In [5]:
test_dict = get_first_n_items(phrase_translations, 3)

In [10]:
phrase_to_modify = ["the_drugs_reaction_was_relatively_mild_thank_goodness"]
subset_dictionary = {k: phrase_translations[k] for k in phrase_to_modify}
print(len(subset_dictionary))

1


In [12]:
generated_audio = upload_phrases_audio_to_gcs(
    phrase_dict=subset_dictionary, bucket_name=config.GCS_PRIVATE_BUCKET, overwrite=True
)

Processing phrases: 100%|██████████| 1/1 [00:06<00:00,  6.99s/it]

phrases/swedish/audio/normal/the_drugs_reaction_was_relatively_mild_thank_goodness.mp3 and phrases/swedish/audio/slow/the_drugs_reaction_was_relatively_mild_thank_goodness.mp3 uploaded to GCS


### Test a sample

In [ ]:
from src.audio_generation import generate_translated_phrase_audio

example_pair = test_dict["i_know_exactly_what_youre_talking_about"]
phrase_tuple = example_pair["english"], example_pair[language_name]

In [ ]:
audio = generate_translated_phrase_audio([phrase_tuple], gender="FEMALE")

In [5]:
phrase_translations["the_drugs_reaction_was_relatively_mild_thank_goodness"]

KeyError: 'the_drugs_reaction_was_relatively_mild_thank_goodness'

### Upload phrases to GCS

In [ ]:
result = upload_phrases_audio_to_gcs(phrase_translations)

In [ ]:
list(result.keys())